# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import warnings
warnings.filterwarnings("ignore")

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Output/cities.csv")

# Display sample data
# Don't need to convert to Imperial according to rubric
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Nanortalik,60.1432,-45.2371,-11.26,82,98,5.09,GL,2023-02-02 22:53:25
1,1,Jamestown,42.0970,-79.2353,1.77,42,0,7.72,US,2023-02-02 22:46:13
2,2,Qaanaaq,77.4840,-69.3632,-17.16,62,14,4.27,GL,2023-02-02 22:57:34
3,3,Kalengwa,-13.4659,25.0027,17.80,95,100,0.55,ZM,2023-02-02 22:59:38
4,4,Ushuaia,-54.8000,-68.3000,15.81,55,0,6.17,AR,2023-02-02 22:50:09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width = 800, 
    frame_height = 600,
    size = "Humidity", 
    scale = 1, 
    color = "City",
    alpha= 0.5
)



# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
#Drop any rows with null values
city_data_df = city_data_df.dropna(how ="any")

city_data_df.count()

City_ID       566
City          566
Lat           566
Lng           566
Max Temp      566
Humidity      566
Cloudiness    566
Wind Speed    566
Country       566
Date          566
dtype: int64

In [24]:
#Find what each column data type is
city_data_df.dtypes 

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [32]:
#Decide ideal temperature range and humidity 
min_temp= 21
max_temp = 30
humidity = 65

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Humidity"] <= humidity)]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,Acapulco,16.8634,-99.8901,29.15,49,1,4.88,MX,2023-02-02 22:57:02
18,18,Geraldton,-28.7667,114.6000,29.23,20,34,8.23,AU,2023-02-02 22:59:42
20,20,Mataura,-46.1927,168.8643,27.92,44,28,2.15,NZ,2023-02-02 22:50:28
40,40,Mount Isa,-20.7333,139.5000,24.87,44,31,6.17,AU,2023-02-02 22:56:01
54,54,Carnarvon,-24.8667,113.6333,28.29,60,21,0.25,AU,2023-02-02 22:57:23
66,66,Port Macquarie,-31.4333,152.9167,29.94,48,3,0.95,AU,2023-02-02 23:00:57
89,89,San Cristobal,7.7669,-72.2250,23.10,46,4,1.42,VE,2023-02-02 22:54:36
93,93,Tuatapere,-46.1333,167.6833,24.44,61,37,0.24,NZ,2023-02-02 23:02:39
117,117,Cabo San Lucas,22.8909,-109.9124,22.56,62,80,1.79,MX,2023-02-02 23:01:03
119,119,Flinders,-34.5833,150.8552,23.54,51,1,4.92,AU,2023-02-02 23:02:48


In [33]:
preferred_cities_df.count()

City_ID       47
City          47
Lat           47
Lng           47
Max Temp      47
Humidity      47
Cloudiness    47
Wind Speed    47
Country       47
Date          47
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = preferred_cities_df[["City", "Country","Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
14,Acapulco,MX,16.8634,-99.8901,49,
18,Geraldton,AU,-28.7667,114.6000,20,
20,Mataura,NZ,-46.1927,168.8643,44,
40,Mount Isa,AU,-20.7333,139.5000,44,
54,Carnarvon,AU,-24.8667,113.6333,60,
66,Port Macquarie,AU,-31.4333,152.9167,48,
89,San Cristobal,VE,7.7669,-72.2250,46,
93,Tuatapere,NZ,-46.1333,167.6833,61,
117,Cabo San Lucas,MX,22.8909,-109.9124,62,
119,Flinders,AU,-34.5833,150.8552,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"

params = {
    "apiKey": geoapify_key, 
    "categories": categories, 
    "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Acapulco - nearest hotel: Hotel del Valle
Geraldton - nearest hotel: Ocean Centre Hotel
Mataura - nearest hotel: No hotel found
Mount Isa - nearest hotel: Ibis Styles
Carnarvon - nearest hotel: No hotel found
Port Macquarie - nearest hotel: The Observatory
San Cristobal - nearest hotel: Terra Sur
Tuatapere - nearest hotel: No hotel found
Cabo San Lucas - nearest hotel: Comfort Rooms
Flinders - nearest hotel: Nova Kiama
Mayor Pablo Lagerenza - nearest hotel: No hotel found
Pau Dos Ferros - nearest hotel: Hotel Anízio
Nioro - nearest hotel: No hotel found
Talcahuano - nearest hotel: Club Recreativo Armada de Chile
Kaitangata - nearest hotel: Hotel South Otago
Rio Cuarto - nearest hotel: Marluc
Ibotirama - nearest hotel: Glória Palace Hotel
El Carmen - nearest hotel: No hotel found
Carora - nearest hotel: Hotel Irpinia
Pouebo - nearest hotel: No hotel found
Diego De Almagro - nearest hotel: Residencial Hambyll
Ewa Beach - nearest hotel: Hampton Inn & Suites Oahu/Kapo

,City,Country,Lat,Lng,Humidity,Hotel Name
14,Acapulco,MX,16.8634,-99.8901,49,Hotel del Valle
18,Geraldton,AU,-28.7667,114.6000,20,Ocean Centre Hotel
20,Mataura,NZ,-46.1927,168.8643,44,No hotel found
40,Mount Isa,AU,-20.7333,139.5000,44,Ibis Styles
54,Carnarvon,AU,-24.8667,113.6333,60,No hotel found
66,Port Macquarie,AU,-31.4333,152.9167,48,The Observatory
89,San Cristobal,VE,7.7669,-72.2250,46,Terra Sur
93,Tuatapere,NZ,-46.1333,167.6833,61,No hotel found
117,Cabo San Lucas,MX,22.8909,-109.9124,62,Comfort Rooms
119,Flinders,AU,-34.5833,150.8552,51,Nova Kiama


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width = 800, 
    frame_height = 600,
    size = "Humidity", 
    scale = 1, 
    color = "City",
    alpha= 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)